In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE


In [1]:
!python -m pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import mysql.connector
import pickle
from sklearn.linear_model import LogisticRegression


conn_config = {
    'host': '192.168.1.185',  # our group's central auth ip address
    'user': 'clientUsers',
    'password': 'ronaldo7',
    'database': 'fl_database'
}
conn = mysql.connector.connect(**conn_config)
cursor = conn.cursor()


cursor.execute(
    "SELECT model_blob FROM central_models WHERE model_id = %s ORDER BY version DESC LIMIT 1",
    (1,)
)

result = cursor.fetchone()
cursor.close()
conn.close()

if result is None:
    raise Exception("Model not found in database.")
model_blob = result[0]
model_blob

b'\x80\x04\x95N\x01\x00\x00\x00\x00\x00\x00\x8c\x1esklearn.linear_model._logistic\x94\x8c\x12LogisticRegression\x94\x93\x94)\x81\x94}\x94(\x8c\x07penalty\x94\x8c\x02l2\x94\x8c\x04dual\x94\x89\x8c\x03tol\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x01C\x94G?\xb9\x99\x99\x99\x99\x99\x9a\x8c\rfit_intercept\x94\x88\x8c\x11intercept_scaling\x94K\x01\x8c\x0cclass_weight\x94\x8c\x08balanced\x94\x8c\x0crandom_state\x94K*\x8c\x06solver\x94\x8c\x04saga\x94\x8c\x08max_iter\x94M\xb8\x0b\x8c\x0bmulti_class\x94\x8c\ndeprecated\x94\x8c\x07verbose\x94K\x00\x8c\nwarm_start\x94\x88\x8c\x06n_jobs\x94J\xff\xff\xff\xff\x8c\x08l1_ratio\x94N\x8c\x10_sklearn_version\x94\x8c\x051.5.2\x94ub.'

In [ ]:
if isinstance(model_blob, str):
    model_blob = model_blob.encode('latin1')  # Or 'utf-8' if needed
model = pickle.loads(model_blob)
model

LogisticRegression(C=0.1, class_weight='balanced', max_iter=3000, n_jobs=-1,
                   random_state=42, solver='saga', warm_start=True)

In [ ]:
model.get_params()

{'C': 0.1,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 3000,
 'multi_class': 'deprecated',
 'n_jobs': -1,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'saga',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': True}

In [4]:
import optuna
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv('../client_dataset/client_3_binary.csv')

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Step 1: Create the engineered features
df['BP_Chol'] = df['HighBP'] * df['HighChol']
df['Age_BMI'] = df['Age'] * df['BMI']
df['HealthCombo'] = df['GenHlth'] + df['PhysHlth']
df['ChronicRiskScore'] = df['HighBP'] + df['HighChol'] + df['HeartDiseaseorAttack'] + df['DiffWalk']

# Step 2: Define features to scale
features_to_scale = ['BP_Chol', 'Age_BMI', 'HealthCombo', 'ChronicRiskScore']

# Step 3: Initialize MinMaxScaler and apply it
scaler = MinMaxScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])


In [7]:

X = df.drop("Diabetes_binary", axis=1)
y = df["Diabetes_binary"]

In [9]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from imblearn.combine import SMOTEENN
from collections import Counter


smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)

In [11]:

# Fit the model on training data
model.fit(X_resampled, y_resampled)

# Now you can predict on test data, for example:
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [12]:
from sklearn.metrics import recall_score, f1_score, classification_report, accuracy_score
print("Classification Report:\n", classification_report(y_train, y_train_pred))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      0.66      0.78     43699
         1.0       0.28      0.83      0.42      7037

    accuracy                           0.69     50736
   macro avg       0.62      0.75      0.60     50736
weighted avg       0.87      0.69      0.73     50736



In [13]:





print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      0.66      0.78     10888
         1.0       0.28      0.82      0.42      1796

    accuracy                           0.68     12684
   macro avg       0.62      0.74      0.60     12684
weighted avg       0.86      0.68      0.73     12684



In [ ]:
import mysql.connector
import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, log_loss


accuracy = accuracy_score(y_test, y_pred)
loss = log_loss(y_test, y_pred)


model_blob = pickle.dumps(model)


report = classification_report(y_test, y_pred, output_dict=True)


macro_f1 = report["macro avg"]["f1-score"]
recall_minority = report["1.0"]["recall"]
f1_minority = report["1.0"]["f1-score"]
f1_majority = report["0.0"]["f1-score"]


fit_status = "good"


model_id = 1
client_id = "3"
round_num = 3


conn = mysql.connector.connect(
    host= "192.168.1.185",  # our group's central auth ip address
    user="centralAuth",
    password="messi10",
    database="fl_database"
)
cursor = conn.cursor()


insert_query = """
INSERT INTO client_updates 
(model_id, client_id, model_blob, accuracy, loss, round_num, macro_f1, recall_minority, f1_minority, f1_majority, fit_status)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


cursor.execute(insert_query, (
    model_id, client_id, model_blob, accuracy, loss, round_num,
    macro_f1, recall_minority, f1_minority, f1_majority, fit_status
))
conn.commit()

print(" Model update inserted successfully with classification metrics.")

cursor.close()
conn.close()
